## Optimization code for PV microgrid:

In [ ]:
### Initialization:

using JuMP, Gurobi, MATLAB


In [ ]:
for my_int = 1:96
    
    #----------------------------------------------------------------------------------------------------
    ### Solver Definition:
    
    m = Model(solver=GurobiSolver(Presolve=1,TimeLimit=600))
        
    #----------------------------------------------------------------------------------------------------
    ### Loading Optimization Parameters:

    mat"generate_data($my_int)"
    
    vars = read_matfile("../../Data/Generated Data/5 - Optimization/scenarios/scen_"*dec(my_int)*".mat")

    interval  = Int64(jvalue(vars["interval"]))  # Current Interval (in day, for which we are optimizing)

    N_EMS   = Int64(jvalue(vars["N_EMS"]))   # num of remaining EMS   intervals (opt. executions)
    N_intra = Int64(jvalue(vars["N_intra"])) # num of remaining Intra intervals (opt. resolution)
    N_dies  = Int64(jvalue(vars["N_dies"]))  # num of diesel generators
    N_scen  = Int64(jvalue(vars["N_scen"]))  # num of different scenations
    N_firm  = Int64(jvalue(vars["N_firm"]))  # num of FIRM Margin Constraint Intervals

    # Upper and Lower bounds:
    P_bat_min = jvalue(vars["P_bat_min"])
    P_bat_max = jvalue(vars["P_bat_max"])
    eta_bat  = jvalue(vars["eta_bat"])
    SOC_min  = jvalue(vars["SOC_min"])
    SOC_max  = jvalue(vars["SOC_max"])
    SOC_init = jvalue(vars["SOC_init"])
    Cap_bat  = jvalue(vars["Cap_bat"])
    P_dies_max = jvalue(vars["P_dies_max"])
    P_dies_min = jvalue(vars["P_dies_min"])
    P_PV_inst = jvalue(vars["P_PV_inst"])

    delta_t = jvalue(vars["delta_t"]) # 1 min

    # f_min = -100000
    f_min = jvalue(vars["f_min"])
    theta_i = jvalue(vars["theta_i"])
    theta_d = jvalue(vars["theta_d"])
    theta_b = jvalue(vars["theta_b"])
    theta_p = jvalue(vars["theta_p"])

    marge_dies = jvalue(vars["marge_dies"])

    L_PV = jvalue(vars["L_PV"]) # generated PV scenarios
    L_C  = jvalue(vars["L_C"]) # generated load scenarios

    # Defining iterators:
    ts  = 1:N_EMS
    ps  = 1:N_intra
    ds  = 1:N_dies
    ss  = 1:N_scen
    m1s = 1:N_firm
    m2s = (N_firm+1):N_EMS
    
    #----------------------------------------------------------------------------------------------------
    ### Variable definitions:

    @variables m begin

        # Battery variables:
        P_bat_min <= P_bat_set[t=ts,p=ps]  <= P_bat_max # Battery setpoint for time t,p (decision var.)
        0 <= P_bat_cha[t=ts,p=ps] <= P_bat_max
        0 <= P_bat_dis[t=ts,p=ps] <= P_bat_max
        X_bat[t=ts,p=ps], Bin                           # Operating mode indicator (decision var.)
                                                        # -> P_bat_set = 2*(X_bat-0.5)*P_bat_abs
        SOC_min <= SOC_bat[t=ts,p=ps] <= SOC_max        # Obtained SOC for time t,p (solution var.)

        # PV Plant:
        0 <= P_PV_set[t=ts,p=ps]  <= P_PV_inst          # PV setpoint variable (decision var.)
        0 <= P_PV[t=ts,p=ps,s=ss] <= L_PV[t,p,s]        # Actual PV generation (solution var.)

        # Diesel Generators:
        ON_dies[t=ts,d=ds], Bin                         # Dies. connectin setpoint (decision var.)
        0 <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max  # Actual dies. generation  (solution var.)
        # P_dies_min <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max # Actual dies. generation  (solution var.)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Variable Initialization (from previous optimization)
    
    if my_int>1

        vars = read_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(my_int-1)*".mat")

        P_bat_set_pre = jvalue(vars["P_bat_set"])[2:N_EMS,ps]
        SOC_bat_pre   = jvalue(vars["SOC_bat"])[2:N_EMS,ps]
        P_PV_set_pre  = jvalue(vars["P_PV_set"])[2:N_EMS,ps]
        P_PV_pre      = jvalue(vars["P_PV"])[2:N_EMS,ps,ss]
        ON_dies_pre   = jvalue(vars["ON_dies"])[2:N_EMS,ds]
        P_dies_pre    = jvalue(vars["P_dies"])[2:N_EMS,ps,ss,ds]
        P_bat_cha_pre = jvalue(vars["P_bat_cha"])[2:N_EMS,ps]
        P_bat_dis_pre = jvalue(vars["P_bat_dis"])[2:N_EMS,ps]
        X_bat_pre     = jvalue(vars["X_bat"])[2:N_EMS,ps]

        setvalue(P_bat_set[1:(N_EMS-1),ps],P_bat_set_pre)
        setvalue(SOC_bat[1:(N_EMS-1),ps],SOC_bat_pre)
        setvalue(P_PV_set[1:(N_EMS-1),ps],P_PV_set_pre)
        setvalue(P_PV[1:(N_EMS-1),ps,ss],P_PV_pre)
        setvalue(ON_dies[1:(N_EMS-1),ds],ON_dies_pre)
        setvalue(P_dies[1:(N_EMS-1),ps,ss,ds],P_dies_pre)
        setvalue(P_bat_cha[1:(N_EMS-1),ps],P_bat_cha_pre)
        setvalue(P_bat_dis[1:(N_EMS-1),ps],P_bat_dis_pre)
        setvalue(X_bat[1:(N_EMS-1),ps],X_bat_pre)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Constraints introduction:
    
    @constraints m begin

        # Constraints:
        #
        # A1 - Diesel power limitation
        # A2 - Diesel power limitation
        # B1 - Power balance equation
        # C1 - PV setpoint limitation
        # D1 - Bat. setpoint equation
        # D2 - Charging    mode const.
        # D3 - Discharging mode const.
        # E1 - FIRM    Diesel reserve margin (separated over scenarios)
        # E2 - RELAXED Diesel reserve margin (mean over scenarios)
        # F1 - 
        # F2 - SOC limitations
        # F3 - 
        # G1 - Min frequency limitation

        A1[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] <= ON_dies[t,d]*(P_dies_max)
        A2[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] >= ON_dies[t,d]*(P_dies_min)

        B1[t=ts,p=ps,s=ss], (P_PV[t,p,s] + sum(P_dies[t,p,s,d] for d=ds) + P_bat_set[t,p] - L_C[t,p,s]) == 0

        C1[t=ts,p=ps,s=ss],  P_PV[t,p,s] <= P_PV_set[t,p]

        D1[t=ts,p=ps], P_bat_set[t,p] == P_bat_cha[t,p] - P_bat_dis[t,p]
        D2[t=ts,p=ps], P_bat_cha[t,p] <= P_bat_max *    X_bat[t,p]
        D3[t=ts,p=ps], P_bat_dis[t,p] <= P_bat_max * (1-X_bat[t,p])

        # E1[t=ts,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E1[t=m1s,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E2[t=m2s,p=ps],  sum(P_dies[t,p,s,d] for d=ds,s=ss)/N_scen <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies

        F1[t=1,p=1],               SOC_bat[t,p] == SOC_init             - P_bat_set[t,p]*delta_t/Cap_bat
        F2[t=2:N_EMS,p=1],         SOC_bat[t,p] == SOC_bat[t-1,N_intra] - P_bat_set[t,p]*delta_t/Cap_bat
        F3[t=1:N_EMS,p=2:N_intra], SOC_bat[t,p] == SOC_bat[t,p-1]       - P_bat_set[t,p]*delta_t/Cap_bat

        G1[t=ts,p=ps], f_min <= theta_i + theta_d*sum(ON_dies[t,d] for d=ds) + theta_b*P_bat_set[t,p] + theta_p*P_PV_set[t,p]

    end

    #----------------------------------------------------------------------------------------------------
    ### Objective function definition:
    
    @objective(m, Max, sum((sum(P_PV[t,p,s] for s=ss)-N_scen*(1-eta_bat)*(P_bat_cha[t,p]+P_bat_dis[t,p])) for t=ts,p=ps));
    
    #----------------------------------------------------------------------------------------------------
    ### Solving Optimization:
    status = solve(m)

    #----------------------------------------------------------------------------------------------------
    ### Saving Data:
    
    write_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(interval)*".mat"; 
        status    = string(status),
        interval  = interval,
        P_bat_set = getvalue(P_bat_set[:,:]), 
        SOC_bat   = getvalue(SOC_bat[:,:]),
        P_PV_set  = getvalue(P_PV_set[:,:]),
        P_PV      = getvalue(P_PV[:,:,:]),
        ON_dies   = getvalue(ON_dies[:,:]),
        P_dies    = getvalue(P_dies[:,:,:,:]),
        P_bat_cha = getvalue(P_bat_cha[:,:]),
        P_bat_dis = getvalue(P_bat_dis[:,:]),
        X_bat     = getvalue(X_bat[:,:]))
    
    # IJulia.clear_output(wait=false)
    println("Current progress: "*dec(my_int)*"/144")
    
    if string(status)=="Infeasible"
        break
    end
    
end